In [1]:
import h5py
import numpy as np
from astropy.io import fits

In [2]:
# We want the average, calibrated shear measurements for each survey
# TXPipe outputs a calibrated, shear catalog (binned_shear_catalog.hdf5)
# Note: we used the config subtract_mean_shear = False in the TXShearCalibration stage

In [3]:
txpath = '/pscratch/sd/j/jjeffers/temp/TXPipe/data/'
datpath = '/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/'

DES-Y3

In [5]:
#non-tomographic = bin_all

with h5py.File(txpath+'desy3/TABLE2/binned_shear_catalog.hdf5','r') as f:
    print(np.average(f['shear/bin_all/g1'][:],weights=f['shear/bin_all/weight'][:]))
    print("unweighted: ",np.mean(f['shear/bin_all/g1'][:]))
    print(np.average(f['shear/bin_all/g2'][:],weights=f['shear/bin_all/weight'][:]))
    print("unweighted: ",np.mean(f['shear/bin_all/g2'][:]))
    print(len(f['shear/bin_all/g2'][:]))

0.0004971728124430973
unweighted:  0.0005463475784292157
8.404160445745847e-05
unweighted:  9.10371899788566e-05
100204026


In [6]:
#checking uncalibrated shear catalog for good measure:

with h5py.File(datpath+'des-y3/shear_catalog_desy3_unmasked_withfakez_v2.h5','r') as f:
    print(np.average(f['shear/mcal_g1'][:],weights=f['shear/weight'][:]))
    print("unweighted: ",np.mean(f['shear/mcal_g1'][:]))
    print(np.average(f['shear/mcal_g2'][:],weights=f['shear/weight'][:]))
    print("unweighted: ",np.mean(f['shear/mcal_g2'][:]))


0.00035588725132367616
unweighted:  0.0003924394377486362
5.9469129003053495e-05
unweighted:  6.556670170768782e-05


KiDS-1000

In [7]:
with h5py.File(txpath+'kids/TABLE2/binned_shear_catalog.hdf5','r') as f:
    Nmask = f['shear/bin_all/dec'][:] > -25.0
    Smask = f['shear/bin_all/dec'][:] < -25.0

    print(np.average(f['shear/bin_all/g1'][:][Nmask],weights=f['shear/bin_all/weight'][:][Nmask]))
    print(np.average(f['shear/bin_all/g2'][:][Nmask],weights=f['shear/bin_all/weight'][:][Nmask]))
    print(len(f['shear/bin_all/g2'][:][Nmask]))
    
    print(np.average(f['shear/bin_all/g1'][:][Smask],weights=f['shear/bin_all/weight'][:][Smask]))
    print(np.average(f['shear/bin_all/g2'][:][Smask],weights=f['shear/bin_all/weight'][:][Smask]))
    print(len(f['shear/bin_all/g2'][:][Smask]))

0.0003292200830319268
0.0008266333013998106
9863348
-0.00010985927605516346
0.0003968082636412022
11298657


HSC-Y3


In [8]:
def rd2tp(ra,dec):
    """Convert ra,dec -> tht,phi"""
    tht = (-dec+90.0)/180.0*np.pi
    phi = ra/180.0*np.pi
    return tht,phi

def tp2rd(tht,phi):
    """Convert tht,phi -> ra,dec"""
    ra  = phi/np.pi*180.0
    dec = -1*(tht/np.pi*180.0-90.0)
    return ra,dec

def pointpick(ra,dec,mask,g1,g2,w):
    nside        = hp.npix2nside(mask.shape[0])
    tht,phi = rd2tp(ra,dec)
    pix = hp.ang2pix(nside,tht,phi)
    idx = np.where(mask[pix]>0)[0]
    #ra,dec = tp2rd(tht[idx],phi[idx]) 
    g1 = g1[idx]
    g2 = g2[idx]
    w = w[idx]
    return g1,g2,w

In [9]:
import healpy as hp

In [10]:
fields = ['GAMA09H','GAMA15H','HECTOMAP','WIDE12H','VVDS','XMM']

for field in fields:
    print(f'{field}:')
    #make mask for the subfield:
    g = fits.open(datpath+f'hsc-y3/shear/{field}_calibrated.fits')
    tht,phi = rd2tp(np.asarray(g[1].data['i_ra']), np.asarray(g[1].data['i_dec']))
    pix     = hp.ang2pix(1024,tht,phi)
    mask= np.bincount(pix,minlength=hp.nside2npix(1024))
    mask[mask>0]=1
    #open calibrated shear catalog
    with h5py.File(txpath+'hsc-y3/outputs/shearsys/TABLE2/binned_shear_catalog.hdf5','r') as h:
        #mask the data and find the objects within the subfield
        g1,g2,weight = pointpick(h['shear/bin_all/ra'][:],h['shear/bin_all/dec'][:],mask,
                                 h['shear/bin_all/g1'][:],h['shear/bin_all/g2'][:],
                                 h['shear/bin_all/weight'][:])
        print(np.average(g1,weights=weight))
        print(np.average(g2,weights=weight))
        print(len(g1))

GAMA09H:
-0.0007092703835266758
0.0007127208883628244
4866052
GAMA15H:
-0.0003166599128251405
-0.0008001954884802261
2573846
HECTOMAP:
-0.001222656622589358
-0.0006032714562021774
2650556
WIDE12H:
0.0009136226139817571
0.00021395515807269327
7278683
VVDS:
0.00026588117760655954
-0.00028226680603456086
6014925
XMM:
-0.0009855226860588742
-0.0007210579224415984
1875168
